<a href="https://colab.research.google.com/github/Vatsalvadi123/EAI6000-Artificial-Intelligence-/blob/master/Week_3_Lab_Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vatsal Vadi
- EAI6000 
- 03/04/2020

In this lab we will train a neural network model to classify images of clothing. 

The lab is designed to be run using Google Colab, but also can be run as a normal jupyter notebook with minor modifications.

This lab uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow.

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Import the Fashion MNIST dataset

In [0]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Examine the dataset size.
1. What are the sizes of each image? 
2. How many categories are there?
3. How many images are there in the test and training sets?
4. What are the range of values for the images? 

In [0]:
print('Train_images', train_images.shape, train_labels.shape)
print('Test_images', (test_images.shape, test_labels.shape))
# summarize pixel values
print('Train_images', train_images.min(), train_images.max(), train_images.mean(), train_images.std())
print('Test_images', test_images.min(), test_images.max(), test_images.mean(), test_images.std())



*   size of each image is 28 * 28. 
*   There is total 10 lables.

*   Training data has 60,000 images. Test data has 10,000 images. 
*   Range is between 0 to 255.





Preprocess the data by scaling the pixel values of all images to fall between 0 and 1.

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

To verify that the data is in the correct format  display the first 10 images from the *training set* and display the class name below each image.


In [0]:
plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Build the model

Using Keras, build a neural network with a single hidden layer consisting of 64 neurons. Remember that you will either have to reshape the image data into 1D vectors, or use a flatten layer at the input.
1. Set up the model layers
2. Compile the model 
3. Print the model summary

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(64, activation='relu'),
     keras.layers.Dense(10)
])

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [0]:
model.summary()

## Train the model

Train the model on the training images for 20 epochs.




In [0]:

cs = model.fit(train_images, train_labels, epochs=20)

Evaluate the accuracy of the model on the test dataset. How does it compare with the training accuracy? Is it likely overfitting or underfitting?

In [0]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

In [0]:
train_loss, train_acc = model.evaluate(train_images,  train_labels, verbose=2)
print('\nTest accuracy:', train_acc)

Accuracvy on test data is 0.88 which is a good number but train set has accuracy of 0.92. That means we have overfitting. 

### Make predictions

The outputs layer of the model contains predictions about the classification for each image. 

In [0]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
class_predictions = model.predict_classes(test_images)
prediction_probabilities = probability_model.predict(test_images)

In [0]:
predictions = probability_model.predict(test_images)

In [0]:
predictions[0]

In [0]:
prediction_probabilities.shape


In [0]:
class_predictions.shape

Examine the shape of the variable 'prediction_probabilities'. How is it different from class_predictions? What do the dimensions correspond to?

Shape of prediction probnability is (10000, 10). while class_prediction has shape of (10000,)

The following functions plot the image and a bar graph of the predicted probabilities. Using these functions, make a plot of the first 25 test images that were incorrectly classified along with their predicted probabilities. Interpret these images - what sort of images tend to be misclassified?

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [0]:
misclassified = np.where(class_predictions != test_labels)
misclassified

In [0]:

count = 0
for i in range(len(test_labels)):
  if np.argmax(prediction_probabilities[i]) != test_labels[i]:
    count += 1
    if count <= 25:
      num_rows = 5
      num_cols = 5
      num_images = num_rows*num_cols
      plt.figure(figsize=(2*2*num_cols, 2*num_rows))
      plt.subplot(num_rows, 2*num_cols+2, 2*count+1)
      plot_image(i, prediction_probabilities[i], test_labels, test_images)
      plt.subplot(num_rows, 2*num_cols+2, 2*count+2)
      plot_value_array(i, prediction_probabilities[i], test_labels)
      plt.tight_layout()
      plt.show()



*  It is seen that mostly same looking items has more chance of being missclassified. like shirts and pullovers, sandels and ankle boots.
*  



Plot a confusion matrix of the predictions. Which misclassifications are the most common?

In [0]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 


In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels,class_predictions)
fig, ax = plt.subplots()
im = ax.imshow(cm)
ax.set_xticks(np.arange(10)), ax.set_yticks(np.arange(10))
ax.set_xticklabels(list(class_names), rotation=45, ha="right")
ax.set_yticklabels(list(class_names))
ax.set_ylabel('True')
ax.set_xlabel('Predicted')
for i in range(100):
    ax.text(int(i/10),i%10,cm[i%10,int(i/10)], ha="center", va="center", color="w")

Mostly misclassified vaues are for shirt  are predicted as puulovers which is 123. 

followed by shirts classified as t-shirt or top.
least classified class is sandle, ankle boot, sandal, coat.
